In [1]:
import pandas as pd
import numpy as np
import random
import os
import re
import json
import pickle

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

from tqdm import tqdm
import progressbar
import h5py

# Model

In [2]:
class AlexNet(nn.Module):
    def __init__(self, img_shape=(1,3,32,32),num_classes=1000,dropout_rate=0):
        super(AlexNet, self).__init__()
        
        #CONV. Layers
        self.conv_stack = nn.Sequential(
            nn.Dropout2d(0.4),
            nn.Conv2d(3, 96, kernel_size=6, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            #original paper uses 2GPU's, so the architecture is split in two main channels which are fed to each GPU
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        input_shape= img_shape
        self.out_conv_dim=self.get_conv_dim(input_shape)
        
        #FFN Layers
        self.fc_stack = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.out_conv_dim, 4096),
            nn.ReLU(inplace=True),
            
            nn.Dropout(dropout_rate),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, num_classes),
        )
        
        
        
    def get_conv_dim(self,shape):
        x = torch.rand(shape)
        x = self.conv_stack(x)
        x=x.view(x.size(0),-1)
        
        return x.size(1)
        
    def forward(self,x):
        x = self.conv_stack(x)
        x = x.view(x.size(0), -1)
        x = self.fc_stack(x)
        
        return x

In [3]:
def weight_init(obj):
    classname = obj.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.kaiming_normal_(obj.weight, mode='fan_out', nonlinearity='relu')
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(obj.weight, 1.0, 0.02)
        torch.nn.init.zeros_(obj.bias)

In [4]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
model=AlexNet(dropout_rate=0.4).to(device)

In [6]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [7]:
get_n_params(model)

34042664

In [27]:
for module in model.modules():
    name=module.__class__.__name__
    if name.find('Conv')!=-1:
        print(name,": CNN")

Conv2d : CNN
Conv2d : CNN
Conv2d : CNN
Conv2d : CNN
Conv2d : CNN


In [9]:
data_dir="/notebooks/datasets/fastai/cifar10/"

In [10]:
data_train=data_dir+'train'
data_test=data_dir+'test'

In [12]:
with open(data_dir+'labels.txt') as file:
    labels=file.read().split('\n')

In [19]:
labels=labels[:-1]

In [11]:
from PIL import Image
import matplotlib.pyplot as plt

In [12]:
category_folders=os.listdir(data_train)

In [13]:
category_images={}
for folder in category_folders:
    category_images[folder]=os.listdir(data_train+'/'+folder)

In [14]:
img=Image.open(data_train+'/bird/'+category_images['bird'][10])

In [15]:
img.size

(32, 32)

In [46]:
img_arr=np.array(img)
img_arr=img_arr.transpose(2,1,0)

In [55]:
img_torch.size()

torch.Size([1, 3, 32, 32])

In [79]:
img_torch=torch.tensor(img_arr.astype(np.float32))
img_torch=img_torch.view(1,img_torch.size(0),img_torch.size(1),img_torch.size(2))

In [ ]:
model(img_torch.to(device))

In [90]:
model.out_conv_dim

2304

In [92]:
os.listdir(data_dir+'train')

['bird',
 'dog',
 'cat',
 'ship',
 'horse',
 'automobile',
 'airplane',
 'frog',
 'deer',
 'truck']

# Dataset

In [16]:
class Cifar10Dataset(Dataset):
    def __init__(self, data_dir, data_size = 0, transforms = None):
        label_folders = os.listdir(data_dir)
        folders_path = [os.path.join(data_dir,x) for x in label_folders]
        
        files=[(os.path.join(path,x),label) for label,path in zip(label_folders,folders_path) for x in os.listdir(path)]
        
        if data_size < 0 or data_size > len(files):
            assert("Data size should be between 0 to number of files in the dataset")
        
        if data_size == 0:
                
            data_size=len(files)
        
        self.data_size = data_size
        self.files = random.sample(files, self.data_size)
        self.transforms = transforms
        self.label_mapping={label:i for i,label in enumerate(label_folders)}
        
    def __len__(self):
        return self.data_size
    
    def __getitem__(self, idx):
        image_address = self.files[idx]
        image = Image.open(image_address[0])
        image=np.array(image).astype(np.float32)
        label_name = image_address[1]
        label = self.label_mapping[label_name]
        
        if self.transforms:
            image = self.transforms(image)

        return image.transpose(2,1,0), label

In [17]:
data_dir

'/notebooks/datasets/fastai/cifar10/'

In [18]:
train_data=Cifar10Dataset(data_dir+'train')

In [19]:
train_data.files[0]

('/notebooks/datasets/fastai/cifar10/train/bird/18727_bird.png', 'bird')

In [20]:
train_data[0][1]

0

In [21]:
train_data.label_mapping

{'bird': 0,
 'dog': 1,
 'cat': 2,
 'ship': 3,
 'horse': 4,
 'automobile': 5,
 'airplane': 6,
 'frog': 7,
 'deer': 8,
 'truck': 9}

In [22]:
train_dl = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=6)

In [23]:
test_data=Cifar10Dataset(data_dir+'test')

In [24]:
test_dl=torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True, num_workers=6)

In [43]:
len(train_dl)

391

# Train

In [26]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    L=len(train_loader)
    for epoch in range(1, epochs + 1):
        model.train() # Make sure that the model is in training mode.

        total_loss = 0
        if epoch==1:
            iterator=tqdm(train_loader)
        else:
            iterator=train_loader
            
        for batch in iterator:
            # get data
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()

            # get predictions from model
            y_pred = model(batch_x)
        
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        if epoch%5==0:
            print("Epoch: {}, Loss: {}".format(epoch, total_loss / L))

In [27]:
num_classes=len(train_data.label_mapping)

In [32]:
model=AlexNet(num_classes=num_classes,dropout_rate=0.4).to(device)

In [33]:
model.apply(weight_init)

AlexNet(
  (conv_stack): Sequential(
    (0): Dropout2d(p=0.4, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(6, 6), stride=(1, 1), padding=(2, 2), bias=False)
    (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kerne

In [34]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [30]:
optimizer=optim.SGD(model.parameters(),lr=0.001,weight_decay=0.0001,momentum=0.0001)
loss_fn=torch.nn.CrossEntropyLoss()

In [46]:
train(model,train_dl,20,loss_fn,optimizer,device)

100%|██████████| 391/391 [00:20<00:00, 18.80it/s]


Epoch: 5, Loss: 0.30565856656302576
Epoch: 10, Loss: 0.29849484149376143
Epoch: 15, Loss: 0.29564154102369344
Epoch: 20, Loss: 0.2818283721461625


In [36]:
def accuracy(model,val_dl):
  model.eval()
  with torch.no_grad():
        accu=0
        for batch in val_dl:
            val_X,val_y=batch
            val_X=val_X.to(device)
            val_y=val_y.to(device)
            x=model(val_X)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_y).sum().item()
    
  accu=accu/len(val_dl.dataset)*100
  return accu

In [40]:
accuracy(model,train_dl)

93.872

In [45]:
accuracy(model,test_dl)

71.96000000000001